In [82]:
import os
import re
import nibabel as nib
import matplotlib.pyplot as plt
import ipyvolume as ipv
import numpy as np
import sklearn.manifold as skm
import time
from patsy import dmatrix
import statsmodels.api as sm

MAX_POINTS = 20941
data = np.load("data/data.npz")

%matplotlib inline

In [91]:
def spline(x, y, df=None,degree=3):
    D = dmatrix("bs(x, df=df, degree=degree, include_intercept=True)",
                {"x": x}, return_type='dataframe')
    fit = sm.GLM(y, D).fit()
    ypred = fit.predict(D)
    return ypred

In [159]:
ind = np.random.randint(0,data["img"].shape[0],1)[0]
# ind=34
# ind=41
show = data["img"][ind,:,:,:,0]
ps = data["points"][ind,:,:]
ps = ps[data["weights"][ind,:]>0.1,:]
print(ps.shape)
ipv.figure()
ipv.volshow(show, lighting=True, level=[1,0.21,0.9], opacity=[0.01,0.1,0.2])
ipv.scatter(ps[:,2],ps[:,1],ps[:,0])
ipv.show()

(5763, 3)


In [154]:
start = time.time()

isomap = skm.Isomap(n_components=1)
ps_embed = isomap.fit_transform(ps)

end = time.time()

print(end - start)

2.569362163543701


In [155]:
pso = ps[np.argsort(ps_embed[:,0]),:]
ts = np.linspace(0,1,num=pso.shape[0])
xs = spline(ts,pso[:,0], df=7)
ys = spline(ts,pso[:,1], df=7)
zs = spline(ts,pso[:,2], df=7)

ipv.figure()
ipv.volshow(show, lighting=True, level=[1,0.21,0.9], opacity=[0.01,0.1,0.2])
ipv.scatter(zs,ys,xs)
# K=13000
# ipv.scatter(pso[:K,2],pso[:K,1],pso[:K,0])
ipv.show()

/usr/local/lib/python3.6/dist-packages/ipyvolume/serialize.py:81: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0]**2 + gradient[1]**2 + gradient[2]**2)
